# Lab 4: One- and two-dimensional data arrays, "netCDF" files, and `xarray` Datasets

## Learning objectives

You should be able to:
- Interpret code using *nested* `for` loops and use them in your own code
- Create and index lists of lists and N-dimensional numpy arrays, including using nested `for` loops
- Make basic heatmaps and contour plots of two-dimensional data using `matplotlib`
- Read and write NetCDF (`.nc`) files using `xarray`
- Extract data values from an `xr.Dataset` using index- and coordinate-based selection

## General Instructions

**Before editing anything:**
  - Click `File` > `Save a Copy in Drive`
  - Click `File` > `Move` > Navigate to `ESS116_Copy` folder > Select folder
  - Click `File` > `Rename` > Rename to `Lab04_LastName_FirstName.ipynb`
  - Verify that you are working in your copy, not the original notebook (which you do not have permission to edit), and are able to save your work!

**While working on the assignment:** As a reminder, you are welcome to talk with your fellow classmates about how to go about completing the assignment, but you must do your own work. I should not see identical notebooks submitted.

**When you are ready to submit:**
- Pin and rename your submissions:
  - Click `File` > `Save and pin revision`
  - Click `File` > `Revision history` > Click the three vertical dots next to the most recent pinned revision (at the very top)
  - Click `Name this version` > Rename to `Submitted assignment` and click `Close`
- Share the link with the instruction team:
  - Click the `Share` button in the top right
  - Uncheck the `Notify people` box
  - Give `Editor` permissions to: **Professor and the TA**
  - Paste the copied URL into the Lab submission form on Canvas

**Deadline:**
- The link to the completed notebook has to be uploaded on Canvas before the start of the next week (by Sunday at 11:59 pm)
- You will get 2 pts extra credit if you upload it before the weekend (by Friday at 11:59 pm)!
- Late reports will be penalized 10% per day after the Sunday deadline.


## Load packages and custom plotting function

This should take about 10 seconds, as we need to install the `cartopy` package for making nice maps.

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr

In [ ]:
# DO NOT EDIT THIS CELL, BUT BE SURE TO RUN ALWAYS RUN IT FIRST!
%pip install cartopy

import matplotlib.pyplot as plt
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LatitudeFormatter, LongitudeFormatter
import cartopy.crs as ccrs

def plot_array_map(ds, extent=[-180, 180, -90, 90], locations=[]):
  """Make a nicely formatted precipitation plot, including coastlines and
  U.S. state borders for reference.

  Optionally change the `extent` keyword argument to zoom into a specific area
  of the globe and the `locations` keyword argument to mark specific locations
  on the map with stars.

  ARGUMENTS
  ---------
  ds [xr.Dataset] -- dataset containing variable "precipitation" that has
    dimensions "lon" and "lat"
  extent [list of float type] -- Box defined by the list of four coordinates:
    [lon_west, lon_east, lat_south, lat_north]
    Default: global extent `[-180, 180, -90, 90]`
  locations [list of instances of Location class] -- see `help(Location)`
    Default: the empty location list `[]`

  RETURNS
  -------
  fig, ax -- matplotlib figure and ax instances

  EXAMPLE
  -------
  # Assuming a precipitation dataset has been assigned to the variable `ds`
  >>> loc_paris = Location("Paris, France", 2.3514, 47.8575)
  >>> plot_array_map(ds, extent=[-25.0, 65.0, 35.0, 72.0], locations=[loc_paris])
  """
  fig, ax = plt.subplots(1, 1, figsize=(12,5), subplot_kw={'projection': ccrs.PlateCarree()})
  ds['precipitation'].plot(
    x="lon",
    y="lat",
    ax=ax,
    transform=ccrs.PlateCarree(),
    cmap=plt.get_cmap("Blues"),
    vmin=0, vmax=150
  )
  s = ax.add_feature(cfeature.STATES, edgecolor='black', facecolor="none", linewidth=0.25, alpha=0.3)
  c = ax.add_feature(cfeature.LAND, edgecolor='black', facecolor="none", linewidth=0.5, alpha=1)
  for l in locations:
    p = ax.plot([l.lon],[l.lat], "*", transform=ccrs.PlateCarree(), label=l.name, markeredgecolor="k", markeredgewidth=0.4, markersize=8)
  s.set_zorder(1)
  c.set_zorder(2)
  ax.set_extent(extent)
  gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linewidth=0.75, color='gray', alpha=0.5, linestyle='--')
  lon_formatter = LongitudeFormatter(zero_direction_label=True)
  lat_formatter = LatitudeFormatter()
  ax.xaxis.set_major_formatter(lon_formatter)
  ax.yaxis.set_major_formatter(lat_formatter);
  if len(locations)>0:
    ax.legend(loc="upper left")

  return fig, ax

class Location():
  """Characterizes a location by its name and geographical coordinates: (lon, lat)."""
  def __init__(self, name, lon, lat):
    """Stores a location's name, longitude (lon), and latitude (lat)

    ARGUMENTS
    ---------
    name [str] -- Name of location
    lon [float] -- longitude of location in degrees east of Greenwich Meridian
                   (valid values from -180.0 to 180.0)
    lat [float] -- latitude of location in degrees north of Equator
                   (valid values from -90.0 to 90.0)

    RETURNS
    -------
    instance of the Location class

    EXAMPLE
    -------
    >>> Location("Paris, France", 2.3514, 47.8575)
    The coordinates of Paris, France are (2.3514ºE, 47.8575ºN).
    """
    self.name = name
    self.lon = lon
    self.lat = lat

  def __repr__(self):
    """Returns an fstring describing the Location instance."""
    return f"The coordinates of {self.name} are ({self.lon}ºE, {self.lat}ºN)."

## Problem 1. Creating and indexing one-dimensional `np.ndarray`s (10 pts)

### Problem 1.1 (1 point)

Use `numpy`'s `linspace` function to assign to the variable `x` an array of $50$ numbers between -1 and 1

In [ ]:
x = np.linspace(-1, 1, 50)


### Problem 1.2 (1 point)

Using the variable `x`, assign the result of $y=x^{2} - 1$ to a new variable `y`.

In [ ]:
y = x**2 - 1


### Problem 1.3 (3 points)

Use a `for` loop to iterate through the indices `i` of `x`. Use an `if` statement within the for loop to print all values of `x[i]` for which `y[i]` is less than $-0.8$.

In [ ]:
for i in range(len(x)):
    if y[i] < -0.8:
        print(x[i])


### Problem 1.4 (3 points)

Use a `for` loop and `if` statement to iterate through the indices `i` of `x` and reassign any values of `y` for which $y<-0.8$ to the corresponding value of $y=-x^2-0.5$.

In [ ]:
for i in range(len(x)):
    if y[i] < -0.8:
        y[i] = -x[i]**2 - 0.5


### Problem 1.5 (2 points)

Use your arrays `y` and `x` computed above to plot the mathematical function $y(x)$ over the interval $x \in [-0.75, 0.75]$. Use the y variable you created in Problem 1.4 and the x variable you have used throughout this problem.

In [ ]:
# Create mask for x in [-0.75, 0.75]
mask = (x >= -0.75) & (x <= 0.75)
plt.plot(x[mask], y[mask])
plt.xlabel('x')
plt.ylabel('y')
plt.title('y(x) over x ∈ [-0.75, 0.75]')
plt.grid(True)
plt.show()


## Problem 2. Does heat kill? (15 pts)

With environmental data, it is often useful to conduct the same analysis for many different places.
However, repeating an analysis for many different places can be time consuming and repetitive. Here
we will analyze and plot data from the six most populous cities in the USA, using `for` loops to make the
analysis more efficient. We will compare actual daily average temperature and daily death counts data
from July 1995, when a particularly deadly heat wave occurred in Chicago.


The directory `/ESS116_Datasets/Lab4/cities_data/` contains 6 files, each containing temperature and mortaility data for a given city. Each of the files has 2 columns and 31 rows. As the column labels suggest, the first column is daily mean temperature in degrees Fahrenheit, and the second column is daily deaths (mortalities) from all causes. Each row index corresponds to a day of the month (e.g., the 4th row, at row index 3, corresponds to July 4, 1995).

**Start by mounting your Google Drive:**

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

### Problem 2.1

**Your goal in this problem:** Write a function `count_hot_days` that takes in a `pd.Dataframe` instance containing daily temperature data and prints a breakdown of how many very hot (>90ºF), hot (80ºF to 90ºF), and normal (<80ºF) days there were.

### Problem 2.1.1 (2 points)

Load in the temperature and mortaility data for Chicago as a `pandas.Dataframe` and assign it to the variable `df`.

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/ESS116_Datasets/Lab4/cities_data/Chicago.csv')


### Problem 2.1.2 (5 points)

Implement your function in the following steps:

- Write the function definition (following Python's function syntax)
- Write a draft Docstring
- In the body of the function:
  - Retrieve the `numpy.ndarray` temperature values from the dataframe instance and assign them to a local variable `temperature`.
  - Create variables that will count the number of each type of day, each starting from `0`.
  - Use a `for` loop and `if-elif-else` statements to iterate through each of the day's temperatures and increment the counters accordingly.
  - Use f-string formatting to print a statement that looks like:
  ```
  There were:
  X very hot days (>90F)
  Y hot days (80-90F)
  Z normal days (<80F)
  ```

In [ ]:
def count_hot_days(df):
    """
    Count and print the number of very hot, hot, and normal days based on temperature data.
    
    ARGUMENTS
    ---------
    df [pd.DataFrame] -- DataFrame containing temperature data in a column named 'Temperature'
    
    RETURNS
    -------
    None (prints the breakdown)
    """
    temperature = df['Temperature'].values
    
    very_hot_count = 0
    hot_count = 0
    normal_count = 0
    
    for temp in temperature:
        if temp > 90:
            very_hot_count += 1
        elif temp >= 80:
            hot_count += 1
        else:
            normal_count += 1
    
    print(f"There were:\n{very_hot_count} very hot days (>90F)\n{hot_count} hot days (80-90F)\n{normal_count} normal days (<80F)")


### Problem 2.1.3 (2 points)

Call your function on the Chicago Dataframe that you previously loaded.

In [ ]:
count_hot_days(df)


### Problem 2.2 (6 points)

Use nested `for` loops to create a figure with twelve panels: a first column with six rows showing the timeseries of daily temperature evolution in each city and a second column showing the corresponding timeseries of mortalities. Use f-strings to title each axis with the corresponding variable and city, and make sure to include x and y axis labels as well.

**Hint:** Start just creating plots of temperature and mortality for a single city, e.g. using the Chicago Dataframe that you already have started analyzing above.

**Hint:** Use an outer `for` loop to loop through the names of the six cities and an inner `for` loop to loop  through the list of variables ("Temperature" and "Mortalities"). Use f-strings to use a city name variable within a path string to load the dataframes.

**Hint:** I have provided you with an initial call to the `plt.subplots` function, which takes as arguments `nrows` and `ncols`, and returns `fig, axes`, to create a figure with an array (or list of list) of axes (each `ax` is an element of the array `axes` returned by `plt.subplots`).

**Hint:** You can use `plt.sca` function to set the current axis for plotting; for example, `plt.sca(axes[1,0])` would set the current axis to the one in the 2nd row and 1st column.

In [ ]:
# <-- DO NOT MODIFY THESE LINES
cities = ["NYC", "LA", "Chicago", "Houston", "Phoenix", "Philadelphia"]
variables = ["Temperature", "Mortalities"]

def days_from_df_index(df):
  """Convert the row index from the temperature/mortaility Dataframes to day-of-month"""
  return np.array(df.index)+1

fig, axes = plt.subplots(6,2,figsize=(9, 12))
# --> DO NOT MODIFY THESE LINES

# <-- Beginning of your code
for i, city in enumerate(cities):
    df_city = pd.read_csv(f'/content/drive/MyDrive/ESS116_Datasets/Lab4/cities_data/{city}.csv')
    days = days_from_df_index(df_city)
    
    for j, var in enumerate(variables):
        plt.sca(axes[i, j])
        plt.plot(days, df_city[var])
        plt.xlabel('Day of Month')
        plt.ylabel(var)
        plt.title(f'{var} - {city}')
        plt.grid(True)
# --> End of your code
plt.tight_layout()

**Discuss:**

**1. What stands out about Chicago?**

[Write answer here]

**2. What do you think could explain this outcomes, compared to the other cities?**

[Write answer here]

## Problem 3. Creating and indexing two-dimensional `np.ndarray`s (10 pts)

### Problem 3.1

Five different ways to create the 3 by 3 identity matrix $I_{3}$.

**Note:** The elements of $I_{3}$ should be floats, not integers.

#### Problem 3.1.1 (1 point)

Create $I_{3}$ manually by calling the `np.array` initiation method on a typed-out list of lists.

In [ ]:
I1 = np.array([[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0]])
I1


#### Problem 3.1.2 (1 point)

Create $I_{3}$ by first creating an array full of zeros (with the `np.zeros` function) and then manually filling in the ones with index-based assignments.


In [ ]:
I2 = np.zeros((3, 3))
I2[0, 0] = 1.0
I2[1, 1] = 1.0
I2[2, 2] = 1.0
I2


#### Problem 3.1.3 (1 point)

Create $I_{3}$ by first creating an array full of zeros (with the `np.zeros` function) and using a for loop and index-based assignment to fill in the ones on the diagonal.

In [ ]:
I3 = np.zeros((3, 3))
for i in range(3):
    I3[i, i] = 1.0
I3


#### Problem 3.1.4 (1 point)

Create $I_{3}$ by first creating an array full of ones (with the `np.ones` function) and using a for loop and index-based assignment to fill in the zeros off of the diagonal.

In [ ]:
I4 = np.ones((3, 3))
for i in range(3):
    for j in range(3):
        if i != j:
            I4[i, j] = 0.0
I4


#### Problem 3.1.5 (1 point)

Create $I_{3}$ using the `np.identity` function.

In [ ]:
I5 = np.identity(3)
I5


### Problem 3.2 (5 points)

Write your own implementation of the `np.identity` function, using a modified version of the method you used in either Problem 3.1.3 or 3.1.4.

Your function should be named `identity`, take in a single argument $N$, and return the $N$ by $N$ identity matrix, $I_{N}$.

**Note:** Don't forget a Docstring!

In [ ]:
def identity(N):
    """
    Create an N by N identity matrix.
    
    ARGUMENTS
    ---------
    N [int] -- Size of the identity matrix (N x N)
    
    RETURNS
    -------
    np.ndarray -- N by N identity matrix with 1.0 on diagonal and 0.0 elsewhere
    """
    I = np.zeros((N, N))
    for i in range(N):
        I[i, i] = 1.0
    return I


Test that your function is working properly by calling it on each of the following cases. Compare your results to those of `np.identity`.

In [ ]:
N=1
print("My function:")
print(identity(N))
print("\nnp.identity:")
print(np.identity(N))
print("\nAre they equal?", np.allclose(identity(N), np.identity(N)))


In [ ]:
N=2
print("My function:")
print(identity(N))
print("\nnp.identity:")
print(np.identity(N))
print("\nAre they equal?", np.allclose(identity(N), np.identity(N)))


In [ ]:
N=7
print("My function:")
print(identity(N))
print("\nnp.identity:")
print(np.identity(N))
print("\nAre they equal?", np.allclose(identity(N), np.identity(N)))


Congratulations, you are now a Python developer!

## Problem 4. Atmospheric River Impacts in Southern California (15 pts)

In early February 2024, California was hit by the devastating impacts of back-to-back *atmospheric rivers*. Atmospheric rivers are bands of warm and moist tropic air that are advected northwards by extra-tropical weather systems.

The GIF below shows how this "precipitable water" gets advected northeastwards from the tropical Pacific towards the California coast. Notice how the precipitable water signal seems to dissipate as it runs into California–this is because it falls out of the atmosphere as torrential rain! **Your goal in this problem is to extract the resulting extreme precipitation rates from global satellite observations.**

<figure>
<left>
<img src='https://www.weather.gov/images/mtr/Storm%20Summary/RainWind_2_3-5_2024/pwat.gif' width=400/>
</left>
</figure>

### Problem 4.1 (1 point)

**Start by mounting your Google Drive:**

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Import `xarray` (using the conventional shorthand `xr`) and use it to open the dataset in `/ESS116_Datasets/Lab4/` that contains daily-averaged precipitation data for February 5th, 2024 (the day of peak rainfall in Southern California) and assign it to the variable `ds`.

**Note:** If you are getting an error when trying to load the dataset and have already double-checked that your Google Drive is correctly mounted, I recommend making a copy of the file and moving it to your personal `ESS116_Copy/` directory, and trying to open that copy instead.


In [ ]:
ds = xr.open_dataset('/content/drive/MyDrive/ESS116_Datasets/Lab4/precipitation_feb5_2024.nc')


The loaded dataset has a `time` dimension with a length of just one (representing the single day of data) in addition to the `lon` and `lat` dimensions.

**Reassign** `ds` to `ds.isel(time=0)` to collapse the time dimension and make `ds` into a more convenient two-dimensional `xr.Dataset`.

In [ ]:
ds = ds.isel(time=0)


### Problem 4.2 (3 points)

Use `matplotlib.pyplot` (already imported as the shorthand `plt` at the top of notebook) to plot the `precipitation` variable in this dataset as a function of longitude (on the x-axis) and latitude (on the y-axis).

**Hint:** The `plt.pcolor` or `plt.pcolormesh` functions expect the 2D array to be plotted to have the second dimension (*columns*) corresponding to the x-axis coordinate and the first dimension (*rows*) corresponding to the y-axis coordinate. Recall that you can use the `transpose` method of an `xr.DataArray` to reverse the order of its dimensions.

**Don't forget to add a title, colorbar, and to label your axis!**

In [ ]:
# Transpose to match expected dimensions (rows = lat, columns = lon)
precip_transposed = ds['precipitation'].transpose('lat', 'lon')
plt.pcolor(ds['lon'], ds['lat'], precip_transposed)
plt.colorbar(label='Precipitation [mm/day]')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Daily Precipitation on February 5, 2024')
plt.show()


### Problem 4.3 (2 points)

It is a bit difficult to extract useful information from the maps above, because we have no visual references to guide us as to where the large precipitation rate are occuring.

Use the custom function `plot_array_map` (defined at the top of this notebook, which uses the [`cartopy` package](https://scitools.org.uk/cartopy/docs/latest/) to add coastlines and state borders to the plot) to make a more informative plot global precipitation patterns on February 5th, 2024.

**Hint:** Use the `help` function to read the function's docstring!

In [ ]:
help(plot_array_map)


In [ ]:
fig, ax = plot_array_map(ds)


### Problem 4.4 (4 points)

**Modify the keyword arguments** of `plot_array_map` to **zoom in** on the atmospheric river hitting Southern California.

Also, **add markers** for the following three locations of interest:
 - San Bernandino, CA
 - Irvine, CA
 - San Diego, CA

**Hint:** Use `help` to look up how to initiate an instance of the `Location` class.

In [ ]:
help(Location)


In [ ]:
# Create Location instances for the three cities
# San Bernardino, CA: approximately 34.1083°N, 117.2898°W (which is -117.2898°E)
loc_san_bernardino = Location("San Bernardino, CA", -117.2898, 34.1083)

# Irvine, CA: approximately 33.6846°N, 117.8265°W (which is -117.8265°E)
loc_irvine = Location("Irvine, CA", -117.8265, 33.6846)

# San Diego, CA: approximately 32.7157°N, 117.1611°W (which is -117.1611°E)
loc_san_diego = Location("San Diego, CA", -117.1611, 32.7157)

# Plot with zoomed extent for Southern California and markers
# Extent: [lon_west, lon_east, lat_south, lat_north]
# Southern California roughly: -120° to -115° longitude, 32° to 35° latitude
fig, ax = plot_array_map(ds, 
                         extent=[-120, -115, 32, 35],
                         locations=[loc_san_bernardino, loc_irvine, loc_san_diego])


### Problem 4.5 (3 points)

Use the `sel` method of an `xr.DataArray` to print the value of the daily-mean precipitation rates at each of the three locations above. Make sure that your print statements clearly identify what location each value corresponds to!

**Hint:** Since the coordinates of the locations above are unlikely to *exactly* match the coordinates of the points in the precipitation dataset, change the setting of the `method` keyword argument of `sel` from the default value of `None` (requiring exact matches) to `nearest` (which instead finds the nearest valid point).

In [ ]:
help(ds['precipitation'].sel)


In [ ]:
# Get precipitation values at each location using sel with method='nearest'
precip_san_bernardino = ds['precipitation'].sel(lon=loc_san_bernardino.lon, lat=loc_san_bernardino.lat, method='nearest')
precip_irvine = ds['precipitation'].sel(lon=loc_irvine.lon, lat=loc_irvine.lat, method='nearest')
precip_san_diego = ds['precipitation'].sel(lon=loc_san_diego.lon, lat=loc_san_diego.lat, method='nearest')

print(f"San Bernardino, CA: {precip_san_bernardino.values:.2f} mm/day")
print(f"Irvine, CA: {precip_irvine.values:.2f} mm/day")
print(f"San Diego, CA: {precip_san_diego.values:.2f} mm/day")


### Problem 4.6 (2 points)

**Describe the spatial distribution of extreme rainfall in Southern California region and discuss the differences between the three locations:**

Write your discussion here.